In [1]:
from keras.layers import Input, Embedding, MultiHeadAttention, LayerNormalization, Dense, Dropout
from keras.models import Model

In [2]:
vocab_size = 10000
sequence_length = 50
embed_dim = 128
num_heads = 4
ff_dim = 512
dropout_rate = 0.1

In [3]:
# Input Layer
inputs = Input(shape = (sequence_length,), name="Input")

In [4]:
# Embedding Layer
x = Embedding(input_dim = vocab_size, output_dim = embed_dim, name = "Embedding")(inputs)

In [6]:
# Multi-Head Self-Attention
attention_output = MultiHeadAttention(num_heads = num_heads, key_dim = embed_dim, name = "MultiHeadAttention")(x, x)

In [7]:
# Add and Norm(Residual Connection and Layer Normalization)
attention_output = LayerNormalization(epsilon=1e-6, name = "Attention_Add_Norm")(x + attention_output)

In [8]:
# Feed-Forward Network (FFN)
ffn = Dense(ff_dim, activation = 'relu', name = "FFN_Dense1")(attention_output)
ffn = Dense(embed_dim, name = "FFN_Dense2")(ffn)

In [9]:
# Add and Norm(Residula connection and Layer Normalization for FFN)
encoder_output = LayerNormalization(epsilon=1e-6, name = "FFN_Add_Norm")(attention_output + ffn)

In [10]:
# Define the model
encoder_model = Model(inputs = inputs, outputs = encoder_output, name='Transformer_Encoder')

In [11]:
# Model summary
encoder_model.summary()

Model: "Transformer_Encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)        │ (None, 50)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Embedding (Embedding)     │ (None, 50, 128)        │      1,280,000 │ Input[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ MultiHeadAttention        │ (None, 50, 128)        │        263,808 │ Embedding[0][0],       │
│ (MultiHeadAttention)      │                        │                │ Embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 50, 128)        │              0 │ Embedding[0][0],       │
│                           │                        │                │ MultiHeadAttention[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Attention_Add_Norm        │ (None, 50, 128)        │            256 │ add[0][0]              │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ FFN_Dense1 (Dense)        │ (None, 50, 512)        │         66,048 │ Attention_Add_Norm[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ FFN_Dense2 (Dense)        │ (None, 50, 128)        │         65,664 │ FFN_Dense1[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 50, 128)        │              0 │ Attention_Add_Norm[0]… │
│                           │                        │                │ FFN_Dense2[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ FFN_Add_Norm              │ (None, 50, 128)        │            256 │ add_1[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,676,032 (6.39 MB)

 Trainable params: 1,676,032 (6.39 MB)

 Non-trainable params: 0 (0.00 B)